# 4. БИБЛИОТЕКИ NUMBA, DATATABLE, BOTTLENECK ДЛЯ УСКОРЕНИЯ ВЫЧИСЛЕНИЙ

## 4.1. Numba

Numba — это библиотека с открытым кодом, которая ускоряет выполнение кода, работающего с массивами NumPy. Ее можно установить командами pip install numba или conda install numba.

Numba использует JIT-компиляцию (Just-In-Time, "на лету"), которая преобразует Python-код в машинный код прямо во время выполнения программы. Это позволяет достичь скорости, сравнимой с языками C или C++, без необходимости переписывать код на другом языке.

Особенности:
1. Интеграция с pandas: После установки Numba в некоторых методах pandas можно указать engine='numba'. Первый запуск будет медленным из-за компиляции, но последующие вызовы выполнятся быстро, так как функция сохраняется в кеше.
2. Декоратор @jit: Можно ускорять свои функции, добавив к ним декоратор `@jit`. Например:
```python
from numba import jit
@jit
def my_function(x, y):
    return x + y
```
  - Если функция использует только поддерживаемые Numba операции, она компилируется в режиме nopython (максимальное ускорение).
  - Если компиляция невозможна (например, из-за несовместимых типов данных), Numba переключается в режим object, но ускорения не будет.
  - Чтобы избежать автоматического переключения в режим object, можно указать `@jit(nopython=True)` или `@njit`.
3. Работа с коллекциями: Списки и множества должны содержать элементы одного типа (например, `[1, 2, 3]` допустимо, а `[1, 2.5]` — нет). Словари поддерживаются, но их нужно создавать через `numba.typed.Dict()`.
4. Пример ускорения кода: Вот функция на чистом Python для вычисления среднего расстояния между значениями. После применения @jit она становится быстрее векторизованных функций NumPy.
```python
def mean_distance(x, y):
    result = 0.0
    for i in range(len(x)):
        result += x[i] - y[i]
    return result / len(x)
```
5. Декоратор @vectorize: Позволяет создавать универсальные функции NumPy из скалярных функций. Такая функция будет работать с массивами так же быстро, как код на C. Например:
```python
@vectorize
def nb_square(x):
    return x ** 2
```
6. Параллельные вычисления: С помощью `@jit(parallel=True)` и `numba.prange()` можно распараллеливать циклы, что значительно ускоряет вычисления. Например:
```python
@jit(parallel=True)
def compute(x):
    s = 0
    for i in prange(x.shape[0]):
        s += x[i]
    return s
```
7. Пример с pandas:
Numba ускоряет методы pandas, например, вычисление скользящего среднего:
```python
roll = series.rolling(10)
%timeit roll.apply(np.mean, engine='numba', raw=True)  # Быстро
%timeit roll.apply(np.mean, raw=True)                  # Медленно
```

## 4.2. Datatable

Datatable — это библиотека Python для быстрой обработки больших наборов данных. Она похожа на pandas, но работает гораздо быстрее, особенно с данными, которые не помещаются в оперативной памяти (RAM). Разрабатывается при поддержке компании H2O.ai. Установить её можно командой: `pip install datatable`

In [1]:
%%time
# Загрузка файла train.csv (291 МБ, ~1.8 млн строк, 34 столбца) Pandas:
import pandas as pd

dataframe = pd.read_csv('train.csv', sep=',')

CPU times: total: 3.16 s
Wall time: 3.4 s


In [6]:
%%time
# Загрузка файла train.csv (291 МБ, ~1.8 млн строк, 34 столбца) Datatable:
import datatable as dt

datatable_df = dt.fread('train.csv', sep=',')

AttributeError: module 'IPython.core.display' has no attribute 'display'